STOCK ANALYZER 

In [2]:
'''
pip install duckdb
'''
from phi.agent import Agent
from phi.tools.yfinance import YFinanceTools
from phi.model.google import Gemini
from phi.tools.csv_tools import CsvTools
from settings import *


csv_agent = Agent(
    name = 'CSV Agent',
    role = 'CSV File analysis',
    tools=[CsvTools(csvs=[stocks_input_data])],
    model=Gemini(id="gemini-2.0-flash-exp"),
    markdown=True,
    show_tool_calls=True,
    instructions=[
        "Retrieve the column 'Ticker Symbols' from the csv file",
    ],
)

#csv_agent.print_response("Read stocks_analysis.csv. Give me the list of stocks", stream='True')

# Finance Agent
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data and interpret trends.",
    model=Gemini(id="gemini-2.0-flash-exp"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions=[
        "Retrieve stock prices, analyst recommendations, and key financial data.",
        "Focus on trends and present the data in tables with key insights."
    ],
    show_tool_calls=True,
    markdown=True,
)

# Team of Agents
agent_team = Agent(
    model=Gemini(id="gemini-2.0-flash-exp"),
    team=[csv_agent, finance_agent, finance_agent],
    instructions=[
        f"Do the following:"
        "1. Ask CSV Agent (agent_0) to read the file stocks_analysis.csv and return the list of ticker symbols from 'Ticker Symbols' column"
        "2. For each stock ask Finance Agent (agent_1) to Analyze stock price movements, analyst recommendations, and any notable financial data of each stock."
        "Highlight key trends or events, and present the data in tables"
    ],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("Analyze stocks from stocks_analysis.csv",stream=True)

#finance_agent.print_response("I am giving you list of stocks: HDFCBANK.NS, TCS.NS. Analyze stock price movements, analyst recommendations, and any notable financial data of each stock. Highlight key trends or events, and present the data in tables.", 
#    stream=True) #To output to Jupyter set stream=True


Output()

INFO     Loading csv file: stocks_analysis

INFO     Running query: SELECT "Ticker Symbols" FROM stocks_analysis